In [2]:
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix 

# display results to 3 decimal points, not in scientific notstion

pd.set_option('display.float_format', lambda x :'%.3f' %x)

user_data = pd.read_table('C:/Users/vishu/Documents/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',
                          header = None , nrows =2e7,
                          names = ['users','musicbrainz-artist-id','artist-name','plays'],
                          usecols = ['users','artist-name','plays'])

user_profile = pd.read_table('C:/Users/vishu/Documents/data/lastfm-dataset-360K/usersha1-profile.tsv',
                             header=None,
                             names =['users','gender','age','country','signup'],
                             usecols =['users','country']    )


In [4]:
user_data.head()

,users,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137.000
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099.000
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897.000
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717.000
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706.000


In [5]:
user_profile.head()

,users,country
0,00000c289a1829a808ac09c00daf10bc3c4e223b,Germany
1,00001411dc427966b17297bf4d69e7e193135d89,Canada
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,Germany
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,Mexico
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,United States


In [6]:
if user_data['artist-name'].isnull().sum() >0:
    user_data = user_data.dropna(axis=0, subset= ['artist-name'])

In [10]:
artist_plays = (user_data.groupby(by =['artist-name'])['plays'].
                sum().
                reset_index().
                rename(columns = {'plays': 'total_artist_plays'})
                [['artist-name','total_artist_plays']])

In [11]:
user_data.head()

,users,artist-name,plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137.000
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099.000
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897.000
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717.000
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706.000


In [13]:
artist_plays.head()

,artist-name,total_artist_plays
0,!!!,9365.000
1,!deladap,193.000
2,!distain,128.000
3,!t.o.o.h.!,101.000
4,#####,6626.000
